# Step 1: Data Acquisition

## Objective
Download historical USD/BRL (BRL=X) exchange rate data from Yahoo Finance and prepare it for feature engineering.

## Process
1. Import required libraries
2. Download daily BRL=X data from 2010 to present
3. Clean and standardize column names
4. Remove unnecessary columns (Volume)
5. Save to `data/raw/` directory

## Output
- `data/raw/BRL_X_raw.csv`: Raw OHLC (Open, High, Low, Close) data with Date index

## Data Source
- Provider: Yahoo Finance
- Ticker: BRL=X (USD/BRL exchange rate)
- Frequency: Daily
- Start Date: 2010-01-01

In [1]:
# Import required libraries
import os
import pandas as pd
import yfinance as yf
from datetime import datetime

print(f"Data acquisition started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Data acquisition started at: 2025-10-25 10:51:49


In [2]:
# Define configuration parameters
TICKER = 'BRL=X'  # USD/BRL exchange rate
START_DATE = '2010-01-01'  # Start from 2010 for sufficient historical data
INTERVAL = '1d'  # Daily data

print(f"Configuration:")
print(f"  Ticker: {TICKER}")
print(f"  Start Date: {START_DATE}")
print(f"  Interval: {INTERVAL}")

Configuration:
  Ticker: BRL=X
  Start Date: 2010-01-01
  Interval: 1d


In [3]:
# Download BRL=X data from Yahoo Finance starting from 2010
data = yf.download(TICKER, start=START_DATE, interval=INTERVAL)

# Reset index to make Date a column, then set it back as index
# This ensures proper handling of the Date column
data.reset_index(inplace=True)
data.set_index('Date', inplace=True)

print(f"Downloaded {len(data)} records from {data.index.min().strftime('%Y-%m-%d')} to {data.index.max().strftime('%Y-%m-%d')}")

C:\Users\TZCYT0\AppData\Local\Temp\ipykernel_27452\800119336.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(TICKER, start=START_DATE, interval=INTERVAL)
[*********************100%***********************]  1 of 1 completed

Downloaded 4116 records from 2010-01-04 to 2025-10-24


In [4]:
# Clean column names
# Yahoo Finance may return multi-index columns or columns with ticker prefix
def clean_column_name(col):
    """
    Standardize column names by:
    - Converting tuples to strings
    - Removing ticker symbols and special characters
    - Removing underscores and spaces
    
    Args:
        col: Column name (can be string or tuple)
    
    Returns:
        Cleaned column name as string
    """
    if isinstance(col, tuple):
        col = '_'.join([str(c) for c in col])
    col = str(col)
    # Remove ticker symbol and special characters
    col = col.replace(TICKER, '').replace('=', '').replace('_', '').replace(' ', '')
    return col

# Show original column names
print(f"Original columns: {list(data.columns)}")

# Apply cleaning
data.columns = [clean_column_name(col) for col in data.columns]
print(f"Cleaned columns: {list(data.columns)}")

# Remove Volume column if present (Volume is not meaningful for forex pairs)
if 'Volume' in data.columns:
    data.drop('Volume', axis=1, inplace=True)
    print("Volume column removed (not applicable for forex data)")

# Display data information
print(f"\nDataset shape: {data.shape}")
print(f"Date range: {data.index.min().strftime('%Y-%m-%d')} to {data.index.max().strftime('%Y-%m-%d')}")
print(f"\nData types:\n{data.dtypes}")
print(f"\nMissing values:\n{data.isnull().sum()}")

Original columns: [('Close', 'BRL=X'), ('High', 'BRL=X'), ('Low', 'BRL=X'), ('Open', 'BRL=X'), ('Volume', 'BRL=X')]
Cleaned columns: ['Close', 'High', 'Low', 'Open', 'Volume']
Volume column removed (not applicable for forex data)

Dataset shape: (4116, 4)
Date range: 2010-01-04 to 2025-10-24

Data types:
Close    float64
High     float64
Low      float64
Open     float64
dtype: object

Missing values:
Close    0
High     0
Low      0
Open     0
dtype: int64


In [5]:
# Display first few rows of the data
print("First 5 rows of the dataset:")
data.head()

First 5 rows of the dataset:


,Close,High,Low,Open
Date,,,,
2010-01-04,1.7190,1.7412,1.6723,1.6930
2010-01-05,1.7370,1.7370,1.6713,1.6713
2010-01-06,1.7315,1.7359,1.6798,1.6798
2010-01-07,1.7389,1.7472,1.6805,1.7242
2010-01-08,1.7320,1.7492,1.6954,1.6954


In [6]:
# Save raw data to data/raw/
raw_data_dir = '../data/raw/'
os.makedirs(raw_data_dir, exist_ok=True)
data.to_csv(os.path.join(raw_data_dir, 'BRL_X_raw.csv'), index=True)
print(f'Raw data saved to {os.path.join(raw_data_dir, "BRL_X_raw.csv")} (Date column saved as index)')

Raw data saved to ../data/raw/BRL_X_raw.csv (Date column saved as index)


In [7]:
# Display a random sample of the data for verification
print("Random sample of 5 records:")
data.sample(5)

Random sample of 5 records:


,Close,High,Low,Open
Date,,,,
2011-08-12,1.6233,1.6253,1.6045,1.6224
2012-05-25,2.0274,2.0360,1.9881,2.0272
2012-07-30,2.0209,2.0365,2.0019,2.0211
2024-12-04,6.0431,6.0718,6.0250,6.0431
2024-08-21,5.4781,5.4838,5.4588,5.4781


In [8]:
# Display basic statistics
print("Descriptive statistics:")
data.describe()

Descriptive statistics:


,Close,High,Low,Open
count,4116.000000,4116.000000,4116.000000,4116.000000
mean,3.677728,3.700861,3.651419,3.674320
std,1.406188,1.416315,1.400953,1.410529
min,1.533700,1.544100,1.525200,1.533700
25%,2.242675,2.255775,2.227225,2.241850
50%,3.684850,3.718600,3.668900,3.692850
75%,5.120125,5.155600,5.081192,5.121725
max,6.300000,6.411100,6.181506,6.298000


## Summary

Data acquisition completed successfully:
- Downloaded historical BRL=X data from Yahoo Finance
- Cleaned and standardized column names
- Removed unnecessary Volume column
- Saved to `data/raw/BRL_X_raw.csv`

## Next Steps
Proceed to `02_feature_creation.ipynb` to:
- Calculate technical indicators 
- Create lagged features
- Generate rolling statistics
- Define target variable (price direction)